In [2]:
df_train = pd.read_csv('./input/train.csv', index_col=0)
df_test = pd.read_csv('./input/test.csv', index_col=0)

In [3]:
y_train = np.log1p(df_train['SalePrice'])
# train_df.drop(['SalePrice'], axis=1, inplace=True)
del df_train['SalePrice']

In [4]:
# axis=0 表示按行合并
all_df = pd.concat([df_train, df_test], axis=0)
all_df.shape

(2919, 79)

In [5]:
all_df.MSSubClass = all_df.MSSubClass.astype(str)

In [6]:
all_dummy_df = pd.get_dummies(all_df)

In [7]:
mean_cols = all_dummy_df.mean()
all_dummy_df.fillna(mean_cols, inplace=True)

In [9]:
numeric_cols = all_df.columns[all_df.dtypes != 'object']
numeric_col_means = all_dummy_df.loc[:, numeric_cols].mean()
numeric_col_std = all_dummy_df.loc[:, numeric_cols].std()
all_dummy_df.loc[:, numeric_cols] = (
    all_dummy_df.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std

In [10]:
dummy_train_df = all_dummy_df.loc[df_train.index]
dummy_test_df = all_dummy_df.loc[df_test.index]

In [11]:
X_train = dummy_train_df
X_test = dummy_test_df

In [13]:
from xgboost import XGBRegressor

xgbr = XGBRegressor()
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [14]:
y_pred = xgbr.predict(dummy_test_df)

Int64Index([1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470,
            ...
            2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919],
           dtype='int64', name='Id', length=1459)

In [19]:
res = pd.DataFrame({'Id': df_test.index, 'SalePrice': np.expm1(y_pred)})

In [20]:
res.to_csv('xgbr-20190323.csv',index=False)